In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = "Langer Text, der in kleinere Abschnitte aufgeteilt werden soll..."

# Konfiguration des Splitters
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # Maximale Abschnittsgröße
    chunk_overlap=20,  # Überlappung zwischen den Abschnitten, um Kontext zu erhalten
    separators=["\n\n", ".", " "]  # Trennzeichen in absteigender Priorität
)

# Aufteilung des Textes
chunks = splitter.split_text(text)

# Ausgabe der Abschnitte
for chunk in chunks:
    print(chunk)


Langer Text, der in kleinere Abschnitte aufgeteilt werden soll...


In [16]:
from sklearn.cluster import KMeans
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

# BERT Modell und Tokenizer laden
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = ("Langer Text, der in semantisch sinnvolle Abschnitte aufgeteilt werden soll. "
        "Hier kommt ein weiterer Satz. Und noch ein Satz. Vielleicht ein letzter Satz. "
        "Ein weiterer Abschnitt beginnt hier. Dieser Abschnitt enthält wichtige Informationen. "
        "Die Informationen müssen erhalten bleiben. Dies ist der Abschluss dieses Abschnitts. "
        "Ein neues Thema wird hier vorgestellt. Dieses Thema ist ebenso wichtig. "
        "Das neue Thema bringt neue Herausforderungen. Hier endet der Text mit einer Zusammenfassung.")

# Text in Sätze aufteilen (oder andere logische Einheiten)
sentences = text.split('.')  # Beispielhafte Vereinfachung

# Entfernen leerer Sätze
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

# Berechne BERT-Embeddings für jeden Satz
embeddings = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy())

# Konvertiere die Liste der Embeddings in ein numpy-Array
embeddings = np.array(embeddings)

# Bestimme die Anzahl der Cluster basierend auf der Anzahl der Sätze
num_clusters = min(len(sentences), 5)

# KMeans Clustering
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(embeddings)

# Cluster-Zuweisungen erhalten
clusters = kmeans.labels_

# Sätze basierend auf Cluster-Zuweisungen gruppieren
clustered_sentences = [[] for _ in range(num_clusters)]
for i, cluster_id in enumerate(clusters):
    clustered_sentences[cluster_id].append(sentences[i])

# Ausgabe der semantisch aufgeteilten Abschnitte
for i, cluster in enumerate(clustered_sentences):
    print(f"Cluster {i + 1}: {' '.join(cluster)}")


Cluster 1: Langer Text, der in semantisch sinnvolle Abschnitte aufgeteilt werden soll Ein neues Thema wird hier vorgestellt Hier endet der Text mit einer Zusammenfassung
Cluster 2: Hier kommt ein weiterer Satz Und noch ein Satz Vielleicht ein letzter Satz Ein weiterer Abschnitt beginnt hier
Cluster 3: Das neue Thema bringt neue Herausforderungen
Cluster 4: Dies ist der Abschluss dieses Abschnitts
Cluster 5: Dieser Abschnitt enthält wichtige Informationen Die Informationen müssen erhalten bleiben Dieses Thema ist ebenso wichtig


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
